# Hollister Ranch parcels

In [1]:
import os
import shapely
import settings
import geopandas as gpd

### Parcels

Read in county parcels file

In [3]:
parcels = gpd.read_file(os.path.join(settings.input_dir, "parcels/AssessorParcels20180823_NoOwners.shp"))

Trim the columns to the ones we need.

In [4]:
trimmed_parcels = parcels[[
    'APN',
    'AgPres',
    'geometry'
]]

Clean up the columns

In [5]:
cleaned_parcels = trimmed_parcels.rename(columns={
    "APN": "apn",
    "AgPres": "is_williamson"
})

In [6]:
cleaned_parcels['apn'] = cleaned_parcels.apn.str.replace("-", "")

Filter down to only those with Williamson Act contracts

In [7]:
williamson_parcels = cleaned_parcels[~gpd.pd.isnull(cleaned_parcels.is_williamson)]

### Owners

Read in the property owners

In [8]:
owners = gpd.pd.read_csv(
    os.path.join(settings.input_dir, "owners/CertifiedSecuredRollWithPropChar.txt"),
    quotechar="^",
    dtype=str
)

Trim it down to the columns we want to keep.

In [9]:
trimmed_owners = owners[[
    'APN',
    'Primaryowr',
    'SecondOwner',
    'ConOwner'
]]

Clean up the columns

In [10]:
cleaned_owners = trimmed_owners.rename(columns={
    "APN": "apn",
    "Primaryowr": "primary_owner",
    "SecondOwner": "secondary_owner",
    "ConOwner": "contract_owner"
}).fillna("")

Merge the parcels with the owners

In [11]:
parcels_with_owners = williamson_parcels.merge(cleaned_owners, on="apn", how="left")

### Hollister Ranch

The ranch is the result of several subdivisions created by Santa Barbara County. We can find them in a shapefile pbulished by the surveyor's office.

In [12]:
subdivisions = gpd.read_file(os.path.join(settings.input_dir, "subdivisions/Parcel_Maps.shp"))

In [13]:
trimmed_subdivisions = subdivisions[[
    'ID',
    'Descript',
    'geometry'
]]

In [14]:
cleaned_subdivisions = trimmed_subdivisions.rename(columns={
    "ID": "subdivision_id",
    "Descript": "subdivision_description",
}).fillna("")

In [15]:
hollister_subdivisions = cleaned_subdivisions[cleaned_subdivisions.subdivision_id.isin([
    'P008/045',
    'P009/045',
    'P009/032',
    'P009/009',
])]

### Hollister Parcels

In [16]:
crs = {'init': 'epsg:4269'}

In [17]:
hollister_parcels = gpd.sjoin(
    parcels_with_owners.to_crs(crs),
    hollister_subdivisions.to_crs(crs),
    how="inner",
    op='intersects'
)

In [18]:
hollister_parcels["geometry"] = hollister_parcels["geometry"].apply(lambda geom: geom.wkb)

hollister_parcels = hollister_parcels.drop_duplicates(["geometry"])

# convert back to shapely geometry
hollister_parcels["geometry"] = hollister_parcels["geometry"].apply(lambda geom: shapely.wkb.loads(geom))

Shave off and add parcels we don't want in the final set.

In [19]:
hollister_parcels = hollister_parcels[~hollister_parcels.apn.isin([
    "083620016",
    "083620015",
    "083570018",
    "083580018",
    "083580020",
    "083580019",
    "083490048",
    "083500025"
])]

### Export

In [20]:
hollister_parcels.to_file(os.path.join(settings.output_dir, "hollister-parcels.shp"))

In [21]:
hollister_parcels.reset_index().drop(columns=['geometry', 'index_right', 'index']).to_csv(
    os.path.join(settings.output_dir, "hollister-parcels.csv"),
    index=False
)